<a href="https://colab.research.google.com/github/dougc333/pytorch/blob/main/Ollama_MMLU_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title #Runtime Info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


Sun Feb 16 02:05:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
#@title # Setup
%cd /content/
!apt install lshw
!curl -fsSL https://ollama.com/install.sh | sh
!git clone --depth 1 https://github.com/chigkim/Ollama-MMLU-Pro
%cd Ollama-MMLU-Pro
!pip install -r requirements.txt


/content
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  pci.ids usb.ids
The following NEW packages will be installed:
  lshw pci.ids usb.ids
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 790 kB of archives.
After this operation, 2,988 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 lshw amd64 02.19.git.2021.06.19.996aaad9c7-2build1 [321 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 usb.ids all 2022.04.02-1 [219 kB]
Fetched 790 kB in 1s (1,164 kB/s)
Selecting previously unselected package lshw.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../lshw_02.19.git.2021.06.19.996aaad9c7-2build1_amd64.deb ...
Unpacking lshw (02.19.git.2021.06.19.996aaad9c7-2bui

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [3]:
#@title # Run server

%env OLLAMA_NUM_PARALLEL=16
%env OLLAMA_FLASH_ATTENTION=1

import threading
import subprocess
server = threading.Thread(target=lambda: subprocess.run(['ollama', 'serve'], check=True), daemon=True)
server.start()
import  time

time.sleep(2)

env: OLLAMA_NUM_PARALLEL=16
env: OLLAMA_FLASH_ATTENTION=1


In [4]:
#@title Download
print("Downloading model...")
!ollama pull llama3:8b-instruct-q8_0 &>/dev/null


In [5]:
#@title Run Test

!rm -rf eval_results
!python run_openai.py \
    --url http://localhost:11434/v1 \
    --model llama3:8b-instruct-q8_0 \
    --category 'computer science' \
    --verbosity 0 \
    --parallel 16


2025-02-16 02:09:02.264307
{
	"comment": "",
	"server": {
		"url": "http://localhost:11434/v1",
		"model": "llama3:8b-instruct-q8_0",
		"timeout": 600.0
	},
	"inference": {
		"temperature": 0.0,
		"top_p": 1.0,
		"max_tokens": 2048,
		"system_prompt": "The following are multiple choice questions (with answers) about {subject}. Think step by step and then finish your answer with \"the answer is (X)\" where X is the correct letter choice.",
		"style": "multi_chat"
	},
	"test": {
		"subset": 1.0,
		"parallel": 16
	},
	"log": {
		"verbosity": 0,
		"log_prompt": true
	}
}
README.md: 100% 10.9k/10.9k [00:00<00:00, 39.7MB/s]
test-00000-of-00001.parquet: 100% 4.15M/4.15M [00:00<00:00, 38.1MB/s]
validation-00000-of-00001.parquet: 100% 45.3k/45.3k [00:00<00:00, 120MB/s]
Generating test split: 100% 12032/12032 [00:00<00:00, 170682.50 examples/s]
Generating validation split: 100% 70/70 [00:00<00:00, 26505.49 examples/s]
assigned subjects ['computer science']
Testing computer science...
100% 410/41